In [126]:
# """A command line version of Minesweeper"""
import random
import re
import time
import numpy as np
from string import ascii_lowercase

from collections import defaultdict
grids_and_bombs = list()
games_count = 0 

def setupgrid(gridsize, start, numberofmines):
    emptygrid = np.array([[0 for i in range(gridsize)] for i in range(gridsize)])

    mines = getmines(emptygrid, start, numberofmines)

    for i, j in mines:
        emptygrid[i][j] = -10

    grid = getnumbers(emptygrid)

    return (grid, mines)


def showgrid(grid):
    gridsize = len(grid)

    horizontal = '   ' + (4 * gridsize * '-') + '-'

    # Print top column letters
    toplabel = '     '

    for i in ascii_lowercase[:gridsize]:
        toplabel = toplabel + i + '   '

    print(toplabel + '\n' + horizontal)

    # Print left row numbers
    for idx, i in enumerate(grid):
        row = '{0:2} |'.format(idx + 1)
#         print 'row'
#         print row
#         print 'i'
#         print i
        for j in i:
            row = row + ' ' + str(j) + ' |'

        print(row + '\n' + horizontal)

    print('')


def getrandomcell(grid):
    gridsize = len(grid)

    a = random.randint(0, gridsize - 1)
    b = random.randint(0, gridsize - 1)

    return (a, b)


def getneighbors(grid, rowno, colno):
    gridsize = len(grid)
    neighbors = []

    for i in range(-1, 2):
        for j in range(-1, 2):
            if i == 0 and j == 0:
                continue
            elif -1 < (rowno + i) < gridsize and -1 < (colno + j) < gridsize:
                neighbors.append((rowno + i, colno + j))

    return neighbors


def getmines(grid, start, numberofmines):
    mines = []
    neighbors = getneighbors(grid, *start)

    for i in range(numberofmines):
        cell = getrandomcell(grid)
        while cell == start or cell in mines or cell in neighbors:
            cell = getrandomcell(grid)
        mines.append(cell)

    return mines


def getnumbers(grid):
    #print 'grid1'
    #print grid
    for rowno, row in enumerate(grid):
        for colno, cell in enumerate(row):
            if cell != -10:
                # Gets the values of the neighbors
                values = [grid[r][c] for r, c in getneighbors(grid,
                                                              rowno, colno)]

                # Counts how many are mines
                grid[rowno][colno] = values.count(-10)
    #print 'grid2'
    #print grid

    return grid


def showcells(grid, currgrid, rowno, colno):
    # Exit function if the cell was already shown
    if currgrid[rowno][colno] != -1:
        return

    # Show current cell
    currgrid[rowno][colno] = grid[rowno][colno]

    # Get the neighbors if the cell is empty
    if grid[rowno][colno] == 0:
        for r, c in getneighbors(grid, rowno, colno):
            # Repeat function for each neighbor that doesn't have a flag
            if currgrid[r][c] != -3:
                showcells(grid, currgrid, r, c)


def playagain():
    choice = raw_input('Play again? (y/n): ')
    return choice.lower() == 'y'


def parseinput(inputstring, gridsize, helpmessage):
    cell = ()
    flag = False
    message = "Invalid cell. " + helpmessage

    pattern = r'([a-{}])([0-9]+)(f?)'.format(ascii_lowercase[gridsize - 1])
    validinput = re.match(pattern, inputstring)

    if inputstring == 'help':
        message = helpmessage

    elif validinput:
        rowno = int(validinput.group(2)) - 1
        colno = ascii_lowercase.index(validinput.group(1))
        flag = bool(validinput.group(3))

        if -1 < rowno < gridsize:
            cell = (rowno, colno)
            message = ''

    return {'cell': cell, 'flag': flag, 'message': message}

def playgame(games_to_play):
    gridsize = 9
    numberofmines = 10
    #print grids_and_bombs
    currgrid = np.array([[-1 for i in range(gridsize)] for i in range(gridsize)])

    grid = []
    flags = []
    starttime = 0

    helpmessage = ("Type the column followed by the row (eg. a5). "
                   "To put or remove a flag, add 'f' to the cell (eg. a5f).")
    showgrid(currgrid)
    print(helpmessage + " Type 'help' to show this message again.\n")

    while True:
        minesleft = numberofmines - len(flags)
        #print 'currgriduptop'
        #print currgrid
        #prompt = raw_input('Enter the cell ({} mines left): '.format(minesleft))
        #result = parseinput(prompt, gridsize, helpmessage + '\n')
        
        #message = result['message']
        #cell = result['cell']
        mystery_squares = np.where(currgrid == -1)
        #print mystery_squares
        mystery_squares = zip(mystery_squares[0], mystery_squares[1])
        n = random.randint(0, len(mystery_squares) - 1)
        cell = mystery_squares[n]
        message = ''
        if cell:
            print('\n\n')
            rowno, colno = cell
            currcell = currgrid[rowno][colno]
            #flag = result['flag']
            flag = False
            #print 'grid'
            #print grid
            if len(grid) ==0:
                grid, mines = setupgrid(gridsize, cell, numberofmines)
                print 'got here'
            if not starttime:
                starttime = time.time()

            if flag:
                # Add a flag if the cell is empty
                if currcell == -1:
                    currgrid[rowno][colno] = -3
                    flags.append(cell)
                # Remove the flag if there is one
                elif currcell == -3:
                    currgrid[rowno][colno] = -1
                    flags.remove(cell)
                else:
                    message = 'Cannot put a flag there'

            # If there is a flag there, show a message
            elif cell in flags:
                message = 'There is a flag there'

            elif grid[rowno][colno] == -10:
                print('Game Over\n')
                showgrid(grid)
                global games_count
                games_count += 1
                if games_count < games_to_play:
                    playgame(games_to_play)
                return

            elif currcell == -1:
                #print 'currgrid1'
                #print currgrid
                showcells(grid, currgrid, rowno, colno)
            else:
                message = "That cell is already shown"

            if set(flags) == set(mines):
                minutes, seconds = divmod(int(time.time() - starttime), 60)
                print(
                    'You Win. '
                    'It took you {} minutes and {} seconds.\n'.format(minutes,
                                                                      seconds))
                showgrid(grid)
                global games_count
                games_count +=1
                if games_count < games_to_play:
                    playgame(games_to_play)
                return
        #print 'currgrid2'
        #print currgrid
        #tuple()]
        if (tuple(grid.reshape((1,gridsize**2))[0] == -10), tuple(currgrid.reshape((1,gridsize**2))[0])) not in grids_and_bombs:
            grids_and_bombs.append((tuple(currgrid.reshape((1,gridsize**2))[0]),(tuple(grid.reshape((1,gridsize**2))[0] == -10))))
        showgrid(currgrid)
        print(message)

<ipython-input-126-4b5f773b5213>:233: SyntaxWarning: name 'games_count' is assigned to before global declaration
  global games_count


In [127]:
playgame(500)

     a   b   c   d   e   f   g   h   i   
   -------------------------------------
 1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 |
   -------------------------------------
 2 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 |
   -------------------------------------
 3 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 |
   -------------------------------------
 4 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 |
   -------------------------------------
 5 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 |
   -------------------------------------
 6 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 |
   -------------------------------------
 7 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 |
   -------------------------------------
 8 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 |
   -------------------------------------
 9 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 | -1 |
   -------------------------------------

Type the column followed by the row (eg. a5). To put or remove a flag, add 'f' to the cell (eg. a

In [128]:
len(grids_and_bombs)

1730

In [11]:
person = raw_input('Enter your name: ')
print('Hello', person)

Enter your name: hello
('Hello', 'hello')


In [10]:
!python --version


Python 2.7.12 :: Anaconda custom (x86_64)


In [34]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])

In [38]:
a.reshape((1,9))[0]

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
import random

In [88]:
tuple(np.random.randint(9, size = 2))

(0, 8)